reading pss:

Inputs:
* filename
* codename

outputs:
* list of pss

assigning metrics to pss:
outputs:
* a pandas dataframe with the right columns
    ps, threshold, size, session_count, mean fitness, difference of fitness, atomicity 


Make plot showing 
* tradeoff between size and mean fitness
* tradeoff between commonality and mean fitness

Then I can decide which pss to convert into the use case. And i can get plots
* partition the pref, based on the fitness (HR 100 maybe?)
* plot the two distributions

In [59]:
# load the pss
import json

import pandas as pd

chosen_filename = r"C:\Users\gac8\PycharmProjects\PSSearch\retail_forecasting_data_collection\local_results\result_06-22-H18'm'27's37.json"
chosen_codename = "[CS CM CC][Len SSize MFit CAtom][G3]"

def read_pss_from_file(filename: str, codename: str):
    with open(filename, "r") as file:
        json_data = json.load(file)
    
    matching_codenames = [results_for_config["results"] for results_for_config in json_data["data"] if results_for_config["config"]["code_name"] == codename]
    results_list = matching_codenames[0]
    
    
    return [(set(item["pattern"]), item["threshold"]) for item in results_list]


pss = read_pss_from_file(chosen_filename, chosen_codename)


In [72]:
from PolishSystem.OperatorsBasedOnSimilarities.similarities_utils import gian_get_similarities
from Gian_experimental.NSGAIICustom.testing_in_vitro.SPRef import OptimisedSPref
# load the dataset

import numpy as np
from retail_forecasting_data_collection.data_file_names import vector_path, fitness_values_path
from PolishSystem.read_data import get_pRef_from_vectors

chosen_pRef = get_pRef_from_vectors(name_of_vectors_file=r"C:\Users\gac8\PycharmProjects\PSSearch\retail_forecasting_data_collection\data\many_hot_vectors_250_random.csv",
                                          name_of_fitness_file=r"C:\Users\gac8\PycharmProjects\PSSearch\retail_forecasting_data_collection\data\fitness_250_random.csv",
                                          column_in_fitness_file=2) # col 4 should be HR 100

chosen_SPref = OptimisedSPref.from_pRef(chosen_pRef)

print(chosen_pRef)

# sanity check on fitness
quantity_of_ones = np.sum(chosen_pRef.fitness_array == 1.0)
quantity_of_zeros = np.sum(chosen_pRef.fitness_array == 0.0)
quantity_of_other = len(chosen_pRef.fitness_array) - quantity_of_zeros - quantity_of_ones
print(f"{quantity_of_zeros = }, {quantity_of_ones = }, {quantity_of_other = }")


similarities = gian_get_similarities(r"C:\Users\gac8\PycharmProjects\PSSearch\retail_forecasting_data_collection\data\cluster_info_250_random.pkl")
print(f"Similarities has shape {similarities.shape}")

PRef with 52639 samples, mean = 7.64
quantity_of_zeros = 0, quantity_of_ones = 0, quantity_of_other = 52639
Similarities has shape (250, 250)


In [49]:
import itertools
from Gian_experimental.NSGAIICustom.evolving_genome_threshold import NCSolutionWithGT
from Core.PRef import PRef


# we find the metrics:

def atomicity_based_on_similarity(ps):
    if len(ps) < 2:
        return -1000
    else:
        linkages = [similarities[a, b] for a, b in itertools.combinations(ps, r=2)]
        return np.average(linkages)
atomicity = atomicity_based_on_similarity

def get_metrics(pattern, threshold, sPRef: OptimisedSPref):
    real_ps = NCSolutionWithGT(pattern, threshold)
    
    matches, non_matches = sPRef.partition_using_threshold(real_ps, threshold)
    
    return {"pattern_str": repr(pattern),
            "threshold": threshold,
            "len": len(real_ps),
            "mf": np.average(matches),
            "mfd": np.average(matches) - np.average(non_matches),
            "atomicity": atomicity_based_on_similarity(real_ps),
            "generality": len(matches) / (len(sPRef.fitnesses))}



example_ps, example_threshold = pss[4]
example_metrics = get_metrics(example_ps, example_threshold, chosen_SPref)
for key, value in example_metrics.items():
    print(key, value)

pattern_str {129, 132, 134, 11, 142, 17, 24, 154, 158, 161, 36, 38, 168, 171, 174, 181, 182, 190, 67, 196, 204, 77, 84, 230, 232, 106, 234, 108, 112, 113, 241, 242, 116, 120, 125}
threshold 3
len 35
mf 0.5690389703574151
mfd 0.0385841252684237
atomicity 0.5987716
generality 0.42362126940101447


In [60]:
import pandas as pd

ps_df = pd.DataFrame([get_metrics(pattern, threshold, chosen_SPref) for pattern, threshold in pss])
display(ps_df)

,pattern_str,threshold,len,mf,mfd,atomicity,generality
0,"{0, 32, 38, 82, 242, 212, 86, 247, 62, 220, 190}",3,11,0.613565,0.076541,0.491311,0.127719
1,"{35, 100, 134, 6, 169, 139, 171, 13, 236, 150,...",3,13,0.628429,0.096775,0.778440,0.156500
2,"{98, 228, 202, 43, 50, 181, 87, 184, 124}",3,9,0.666984,0.130608,0.517781,0.079808
3,"{66, 109, 82, 213, 247, 27}",3,6,0.544865,-0.002006,2.182548,0.035145
4,"{134, 167, 211, 212, 52, 220, 29}",3,7,0.661286,0.122873,0.427432,0.068257
...,...,...,...,...,...,...,...
88,"{103, 171, 145, 82, 116, 53, 217, 218, 31, 223}",3,10,0.609098,0.069109,0.659067,0.098558
89,"{196, 7, 137, 180, 85, 120, 249}",3,7,0.743706,0.202876,0.409169,0.029427
90,"{69, 233, 47, 181, 25, 188}",3,6,0.620232,0.080707,0.933120,0.090142
91,"{66, 228, 132, 133, 181, 27}",3,6,0.696833,0.154575,-0.004083,0.029389


In [70]:

len_is_small = ps_df["len"] < 8
mfd_is_high = ps_df["mfd"] > 0.0
generality_is_high = ps_df["generality"] >= 0.0
atomicity_is_positive = ps_df["atomicity"] > 0

acceptable_ps_df = ps_df[len_is_small & mfd_is_high & generality_is_high & atomicity_is_positive]
display(acceptable_ps_df)


winner_pattern, winner_threshold = ({139, 204, 86, 184, 57, 221, 62, 63}, 3)
print(winner_pattern, winner_threshold)


,pattern_str,threshold,len,mf,mfd,atomicity,generality
4,"{134, 167, 211, 212, 52, 220, 29}",3,7,0.661286,0.122873,0.427432,0.068257
6,"{134, 9, 237, 13, 143, 191}",3,6,0.703586,0.160615,1.229850,0.023842
8,"{71, 208, 248, 155, 93}",3,5,0.689365,0.146195,0.652119,0.024829
13,"{204, 146, 181, 86, 60}",3,5,0.708333,0.165999,0.517477,0.026900
29,"{38, 13, 79, 49, 177, 155}",3,6,0.694507,0.152888,2.284779,0.033891
31,"{3, 211, 180, 119, 92, 190}",3,6,0.675231,0.132244,1.241469,0.028838
32,"{171, 53, 24, 25, 156, 95}",3,6,0.667449,0.124687,1.937239,0.032390
52,"{32, 42, 145, 209, 188}",3,5,0.634059,0.091656,1.022864,0.047968
61,"{68, 13, 82, 116, 184, 249}",3,6,0.619625,0.076795,1.121531,0.051692
72,"{100, 169, 19, 247, 188}",3,5,0.637017,0.094985,0.647536,0.050191


{139, 204, 86, 184, 57, 221, 62, 63} 3


In [71]:
winner_real_ps = NCSolutionWithGT(winner_pattern, winner_threshold)
    
# the fitness column should be 4 for this
matches, non_matches = chosen_SPref.partition_using_threshold(winner_real_ps, winner_threshold)

print(matches)
print(non_matches)

## here is the explanation
print(f"If you match at least {winner_threshold} clusters out of \n{winner_pattern}, then")
print(f"The hit rate is {np.average(matches)}, whereas the average overall is {np.average(chosen_SPref.fitnesses)}")
print(f"This pattern occurrs {len(matches) / len(chosen_SPref.fitnesses)} of the time")



[1. 1. 1. ... 0. 0. 1.]
[1. 1. 1. ... 0. 1. 1.]
If you match at least 3 clusters out of 
{139, 204, 86, 184, 57, 221, 62, 63}, then
The hit rate is 0.6401608219790038, whereas the average overall is 0.5467999012139289
This pattern occurrs 0.08505100780789909 of the time


In [93]:
import os

# the fitness column should be 2 for this
winner_real_ps = NCSolutionWithGT(winner_pattern, winner_threshold)
    
# the fitness column should be 4 for this
matches, non_matches = chosen_SPref.partition_using_threshold(winner_real_ps, winner_threshold)

import matplotlib.pyplot as plt
import numpy as np

def make_distribution_comparison_plot(values_a, values_b, bins, x_label, y_label,
                                      title, legend_names, file_destination, figsize,
                                      normalise=False):
    """
    Plots two overlapping histograms comparing the distributions of two datasets.
    When normalise=True, shows frequency percentages on y-axis.
    """
    plt.figure(figsize=figsize)

    if normalise:
        # Use the same bin edges for both histograms
        counts_a, bin_edges = np.histogram(values_a, bins=bins, density=True)
        counts_b, _ = np.histogram(values_b, bins=bin_edges, density=True)

        # Convert density to percentages
        counts_a *= 100
        counts_b *= 100

        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        width = bin_edges[1] - bin_edges[0]

        # Plot overlapping bars at the same positions
        plt.bar(bin_centers, counts_a, width=width, alpha=0.5,
                label=legend_names[0], color='steelblue', edgecolor='black')
        plt.bar(bin_centers, counts_b, width=width, alpha=0.5,
                label=legend_names[1], color='darkorange', edgecolor='black')

        plt.ylabel('Frequency (%)')
    else:
        plt.hist(values_a, bins=bins, alpha=0.6,
                 label=legend_names[0], color='steelblue', edgecolor='black')
        plt.hist(values_b, bins=bins, alpha=0.6,
                 label=legend_names[1], color='darkorange', edgecolor='black')
        plt.ylabel(y_label)

    plt.xlabel(x_label)
    plt.xlim((0, 25))
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    if file_destination:
        plt.savefig(file_destination)
    else:
        plt.show()

    plt.close()


case_study_folder = r"C:\Users\gac8\PycharmProjects\PSSearch\retail_forecasting_data_collection\Case study"
normalised_distributions_destination = os.path.join(case_study_folder, "normalised_distribution.pdf")
make_distribution_comparison_plot(matches, non_matches, 
                                  bins=30, 
                                  x_label="$\mathregular{f_2}\ value$", 
                                  y_label="frequency", 
                                  title="Comparison of fitness distributions", 
                                  legend_names=("with pattern", "without pattern"), 
                                  file_destination=normalised_distributions_destination, 
                                  figsize=(6, 3), 
                                  normalise=True)
